# Overview

We will use two separate data sources, the FIFA Soccer ranking and the World Cup 2018 Player Comparisons and perform ETL to allow us to join on the country and try to establish if the ranking of a country/team can be predicted by the style of its players.

In [ ]:
# Dependencies
import pandas as pd
import datetime
from sqlalchemy import create_engine, inspect

# Extract

### The FIFA Ranking Data Set

The world football governing body FIFA has been ranking international teams since 1992. They are ranked based on their game results with the most successful teams being ranked highest. This dataset contains all available FIFA men’s international soccer rankings from August 1993 to April 2018.
(Source https://en.wikipedia.org/wiki/FIFA_World_Rankings)

### The Players Stats

Each metric is measured on a per-match basis, and for each metric we calculate a z-score — the number of standard deviations above or below average for that World Cup. The similarity between players’ performances is based solely on the average between each of their 16 z-scores — in other words, comparisons match players who are good at similar parts of the game, but the model ignores details like a player’s age or position. Players must have played at least 30 minutes in a given World Cup to be included. Play styles are generated through k-means clustering.

Only successful crosses, tackles, passes, take-ons and headers are counted, and tackles, interceptions and blocks are adjusted for the time of possession that the player's team had during each match. Progressive passes and dribbles advance the ball at least 10 yards toward the opponent's goal or into their box. Expected goals is the number of goals that our model thinks an average player would score based on the quality and quantity of shots taken, and non-shot expected goals is an estimate of the number of goals an average player’s non-shooting actions — passes, take-ons, interceptions, tackles and headers — would generate for his team.

All metrics here were calculated by FiveThirtyEight using data from the sports analytics company Opta and may differ from similar metrics at other sites. Numbers will update after each match through the end of the 2018 World Cup. (Source https://projects.fivethirtyeight.com/world-cup-comparisons/nicolas-otamendi-2018/)
![Player Stats](Resources/players_visual.png)

### Sources
We will import both sheets from the Resources folder

In [ ]:
# Import FIFA Ranking csv downloaded from .....
fifa_ranking = "Resources/fifa_ranking.csv"
fifa_ranking_df = pd.read_csv(fifa_ranking)
fifa_ranking_df.head()

In [ ]:
# Import Player Statistics from ....
world_cup_comparisons = "Resources/world_cup_comparisons.csv"
world_cup_df = pd.read_csv(world_cup_comparisons)
world_cup_df.head()

# Transform

We Check for Unique Year values to see if both sheets match then Do the same with Fifa ranking. We then Create a column with just the year value for FIFA Ranking. We Remove the years that don't match and Review the data. We need to remove columns from the Players stats to only keep Attack and Defence Stats. We rename the columns to make sense and help with joining the other table and reset Index and select only columns we need.

In [ ]:
# Check for Unique Year values to see if both sheets match
world_cup_df.season.unique()

In [ ]:
# Do the same with Fifa ranking...oops they don't match
fifa_ranking_df.rank_date.unique()

In [ ]:
# Create a column with just the year value for FIFA Ranking
fifa_ranking_df['year'] = pd.DatetimeIndex(fifa_ranking_df['rank_date']).year

In [ ]:
# Check again - looks good
fifa_ranking_df.year.unique()

In [ ]:
# Remove the years that don't match in the FIFA data set
fifa_match = fifa_ranking_df.loc[fifa_ranking_df['year'].isin([2018, 2014, 2010, 2006, 2002, 1998, 1994])]
fifa_match.year.unique()

In [ ]:
# Do the same for the Player Statistics Data set
world_cup_matched = world_cup_df.loc[world_cup_df['season'] >= 1994]
world_cup_matched.season.unique()

In [ ]:
# Review the data for Players Stats

world_cup_matched.head()

In [ ]:
# Review the data for the FIFA Rankings
fifa_match.head()

In [ ]:
# We need to remove columns from the Players stats to only keep Attack and Defence Stats:

player_stats = world_cup_matched[['player','season', 'team', 'goals_z', 'xg_z', 'crosses_z', 'boxtouches_z', 
                                      'tackles_z', 'interceptions_z','clearances_z','blocks_z']]
player_stats.head()

In [ ]:
# Rename the columns to make sense and help with joining the other table
player_stats = player_stats.rename(columns={'season': 'year', 
                                            'goals_z': 'goals',
                                            'xg_z':'expected_goals',
                                            'crosses_z':'crosses',
                                            'boxtouches_z':'box_touches',
                                            'tackles_z':'tackles',
                                            'interceptions_z':'interceptions',
                                            'clearances_z':'clearances',
                                            'blocks_z':'blocks'
                                           })
player_stats.head()

In [ ]:
# Reset Index for Players Stats
player_stats = player_stats.reset_index(drop=True)
player_stats.head()

In [ ]:
# Select helpful columns from FIFA rnking
fifa_ranking = fifa_match[['rank', 'country_full','total_points', 
                                  'rank_change', 'cur_year_avg', 'last_year_avg', 'two_year_ago_avg', 'three_year_ago_avg', 
                                  'confederation', 'year']]

fifa_ranking.head()

In [ ]:
# Rename columns for FIFA Ranking

fifa_ranking = fifa_ranking.rename(columns={'country_full': 'team'})
fifa_ranking.head()

In [ ]:
# Reset Index for FIFA Ranking
fifa_ranking = fifa_ranking.reset_index(drop=True)
fifa_ranking.head()

# Load
We need to analyze data types then Create a unique id to hold the primary key for the target tables.

In [ ]:
# First analyze data types
player_stats.dtypes

In [ ]:
fifa_ranking.dtypes

In [ ]:
# Create a unique id to hold the primary key for the target table
fifa_ranking['ranking_id'] = fifa_ranking.index
fifa_ranking.head()

In [ ]:
# Move it to the front because it looks better
fifa_ranking = fifa_ranking[['ranking_id','rank','team','total_points','rank_change',
                             'cur_year_avg','last_year_avg','two_year_ago_avg',
                             'three_year_ago_avg','confederation','year'
                            ]]
fifa_ranking.head()

In [ ]:
# Prepare a Primary Key for the Players Stats
player_stats['player_id'] = player_stats.index
player_stats = player_stats[['player_id', 'player','year','team','goals',
                             'expected_goals','crosses','box_touches','tackles',
                             'interceptions','clearances','blocks'
                            ]
                           ]
player_stats.head()

## HALT!! - Any code below this line requires the database and tables to be created and defined. Read detailed instructions in Readme
You should get at database looking like this: 

![ERD](Resources/erd.png)

#### Make sure to replace postgres, bootcamp and 5432 in the line below with your own username, password and port.
```
**postgres**:**bootcamp**@localhost:**5432**/soccer_db
```

In [ ]:
# Input your pg admin username and password Connect to the database
connection_string = "postgres:bootcamp@localhost:5432/soccer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Load FIFA Ranking into the tables

fifa_ranking.to_sql(name='fifa_ranking', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from fifa_ranking', con=engine).head()

In [ ]:
# Load Players Stats into the table

player_stats.to_sql(name='player_stats', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from player_stats', con=engine).head()

# Exploration
If we wanted to use the prepared data to perform and analysis on how player styles affect team rankings, we can perform certain joins. The `joins.sql` file in the root has a couple of examples.